# Host Annotable web app as Bokeh server

Make a bokeh plot in a Jupyter notebook and serve it up to a remote browser.

ToDo:
----
- Make a basic `bokeh` plot servable
- Make a `holoviews` plot servable
- Make a `hvplot` with gridded data and a pen servable
- Test whether doodles are synched back into the Notebook

## Setup

The following requirements are needed for serving up Bokeh plots from JupyterHub

`$ pip install nbserverproxy && jupyter serverextension enable --py nbserverproxy`

## Basic `bokeh` plot served from Jupyter Lab

## `holoviews` plot served from Jupyter Lab

In [ ]:
import hvplot as hplt
import xarray as xr
import hvplot.xarray
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import cartopy.feature as cf
from holoviews.streams import FreehandDraw

In [ ]:
air_temp = xr.tutorial.open_dataset('air_temperature').load().air.isel(time=0)
print("Loaded dataset")

In [ ]:
coastlines = gv.feature.coastline

proj = ccrs.Orthographic(-90, 30)

air_plot = air_temp.hvplot.quadmesh(
   'lon', 'lat', projection=proj, project=True, global_extent=True,
    width=600, height=540, cmap='viridis', rasterize=True, dynamic=False)

print("Made plot elements")

In [ ]:
warm_front = gv.Path([]).opts(color='red', line_width=9)
warm_front_pen = FreehandDraw(source=warm_front)

warning_orange = gv.Polygons([]).opts(line_color='orange', line_width=9, 
                                      fill_color='orange', fill_alpha=0.6)
warning_orange_tool = FreehandDraw(source=warning_orange)

print("Made pen elements")

In [ ]:
plot = air_plot * coastlines * warm_front * warning_orange

print("Combined elements")

## Deploy Jupyter notebook as `bokeh.server`

In [ ]:
doc = hv.renderer('bokeh').server_doc(plot)
doc.title = 'Annotable Bokeh App - From Jupyter Notebook'

print("Serving document")

To run this notebook as a Bokeh app, run the following in a terminal:

`$ bokeh serve --show bokeh_server.ipynb`

## Deploy plot as a `bokeh.app`

In [ ]:
renderer = hv.renderer('bokeh')
print(renderer)

In [ ]:
app = renderer.app(air_plot * coastlines * warm_front)
print(app)

In [ ]:
from bokeh.server.server import Server

# server = Server({'/app': app}, port=0)
server = Server({'/app': app}, port=7777, allow_websocket_origin=['pangeo.informaticslab.co.uk'])

In [ ]:
server.start()
server.show('/app')

In [ ]:
warm_front_pen.element.data

In [ ]:
server.stop()

In [ ]:
warm_front_pen.element.data

In [ ]:
warm_front_pen.element

## Define annotable function

In [ ]:
def make_annotable(plot, port=0, websocket_origin='pangeo.informaticslab.co.uk', url_path='annotable'):
    import holoviews as hv
    from bokeh.server.server import Server
    import os
    from IPython.core.display import display, HTML
    
    renderer = hv.renderer('bokeh')
    app = renderer.app(plot)
    server = Server({f'/{url_path}': app}, port=port, allow_websocket_origin=[websocket_origin])

    prefix = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    url = f"https://{websocket_origin}{prefix}proxy/{server.port}/{url_path}"
    display(HTML(f'<a href={url}>{url}</a>'))
    
    return server

In [ ]:
annotable = make_annotable(plot)

In [ ]:
annotable.start()

In [ ]:
warm_front_pen.element()

In [ ]:
warning_orange_tool.element()

In [ ]:
annotable.stop()